In [1]:
import numpy as np
import torch
import torch.nn.functional as F
from prep import get_data
from torch import nn
from kfold_experiments import kfold_normal_training_softgedt, kfold_adversarial_training_softgedt,kfold_normal_training_neural_networks, kfold_adversarial_training_neural_networks

# Create dataset

In [2]:
# load dataset
filename = 'authent'

# Preprocess the data without standardizing
data, ncat,dtypes = get_data(filename)

# create editability vector
non_editable_vector = torch.tensor([1 for i in range(data[:,:-1].shape[1])])

# record datatypes
dtypes[dtypes=='object'] = 'pos_int'
dtypes[dtypes=='float64'] = 'float'

datatypes = dtypes[:-1].to_numpy().astype('object')
datatypes[(datatypes=='int64') & np.min(data[:,:-1]>=0,axis=0)]='pos_int'
datatypes[(datatypes=='float64') & np.min(data[:,:-1]>=0,axis=0)]='pos_float'
datatypes = datatypes.tolist()

# Normal Training

In [3]:
kfold_normal_training_softgedt(filename, data, ncat, non_editable_vector, datatypes, include_sum_weight = True)

experiment: 0
Fold: 0
epoch: 0
train acc: tensor(0.9951) valid acc: tensor(1.) trigger times: 0
epoch: 1
train acc: tensor(0.9976) valid acc: tensor(1.) trigger times: 0
epoch: 2
train acc: tensor(0.9988) valid acc: tensor(1.) trigger times: 1
epoch: 3
train acc: tensor(0.9988) valid acc: tensor(1.) trigger times: 2
epoch: 4
train acc: tensor(0.9976) valid acc: tensor(1.) trigger times: 3
Early Stopping
SoftGeDT Accuracy on Training set: tensor(0.9988)
SoftGeDT Accuracy on Validation set: tensor(1.)
SoftGeDT Accuracy on Test set: tensor(1.)
Average Accuracy on Test set: tensor(1.)
Original Accuracy on Test set: tensor(0.9964)
Average Original Accuracy on Test set: tensor(0.9964)
Original HardGeDT Accuracy on Test set: tensor(0.9709)
Average Original HardGeDT Accuracy on Test set: tensor(0.9709)
ADT error, epsilon:0.1: 0.02909090909090909
Average ADT error, epsilon:0.1 0.02909090909090909
HardGeDT Adversarial Error: tensor(0.0655)
HardGeDT Average Adversarial Error: tensor(0.0655)
Fold:

# Adversarial Training

In [4]:
kfold_adversarial_training_softgedt(filename, data, ncat, non_editable_vector, datatypes, include_sum_weight=True)

Fold: 0
epoch: 0
train error: 0.024330900243309004 train loss: 0.10592060204439643 validation error 0.025454545454545455
epoch: 1
train error: 0.023114355231143552 train loss: 0.06951309286983773 validation error 0.04
epoch: 2
train error: 0.021897810218978103 train loss: 0.06292216268619363 validation error 0.025454545454545455
epoch: 3
train error: 0.014598540145985401 train loss: 0.05642434465882782 validation error 0.02909090909090909
epoch: 4
train error: 0.20194647201946472 train loss: nan validation error 0.43272727272727274
SoftGeDT Adversarial Error on Training set: 0
SoftGeDT Adversarial Error on Validation set: 0
SoftGeDT Accuracy on Test set: tensor(0.9927)
Average Accuracy on Test set: tensor(0.9927)
ADT error, epsilon:0.1: 0.02181818181818182
Average ADT error, epsilon:0.1 0.02181818181818182
Fold: 1
epoch: 0
train error: 0.038929440389294405 train loss: 0.13273046666036148 validation error 0.025454545454545455
epoch: 1
train error: 0.031630170316301706 train loss: 0.0943

# Simple Neural Network

In [3]:
class Net(nn.Module):
    def __init__(self, input_dim):
        super(Net, self).__init__()
        self.layer1 = nn.Linear(input_dim, 80).double()
        self.layer2 = nn.Linear(80, 80).double()
        self.layer3 = nn.Linear(80, 80).double()
        self.layer4 = nn.Linear(80, 2).double()
        self.dropout = nn.Dropout(0.5)
    def forward(self, x):
        x = F.relu(self.dropout(self.layer1(x)))
        x = F.relu(self.dropout(self.layer2(x)))
        x = F.relu(self.dropout(self.layer3(x)))
        x = self.layer4(x)
        return x

## 5-fold cross validation normal training

In [9]:
kfold_normal_training_neural_networks(nn, filename, data, ncat, non_editable_vector, datatypes)

Fold: 0
epoch: 0
train acc: tensor(0.9465) valid acc: tensor(1.) trigger times: 0
epoch: 1
train acc: tensor(0.9964) valid acc: tensor(0.9927) trigger times: 0
epoch: 2
train acc: tensor(0.9988) valid acc: tensor(0.9927) trigger times: 1
epoch: 3
train acc: tensor(0.9866) valid acc: tensor(0.9855) trigger times: 2
epoch: 4
train acc: tensor(0.9988) valid acc: tensor(1.) trigger times: 3
epoch: 5
train acc: tensor(1.) valid acc: tensor(1.) trigger times: 0
epoch: 6
train acc: tensor(1.) valid acc: tensor(1.) trigger times: 1
epoch: 7
train acc: tensor(1.) valid acc: tensor(1.) trigger times: 2
epoch: 8
train acc: tensor(1.) valid acc: tensor(1.) trigger times: 3
Early Stopping
FNN Accuracy on Training set: tensor(1.)
FNN Accuracy on Validation set: tensor(1.)
FNN Accuracy on Test set: tensor(0.9927)
Average Accuracy on Test set: tensor(0.9927)
Original Accuracy on Test set: tensor(0.4800)
Average Original Accuracy on Test set: tensor(0.4800)
ADT error, epsilon:0.1: 0.03636363636363636
A

## 5 fold cross validation adversarial training

In [10]:
kfold_adversarial_training_neural_networks(Net, filename, data, ncat, non_editable_vector, datatypes)

Fold: 0
epoch: 0
train error: 0.09975669099756691 train loss: 0.23036077888763118 validation error 0.01818181818181818
epoch: 1
train error: 0.046228710462287104 train loss: 0.13205015884815927 validation error 0.01090909090909091
epoch: 2
train error: 0.03527980535279805 train loss: 0.09509670732289738 validation error 0.04
epoch: 3
train error: 0.024330900243309004 train loss: 0.0635627447050348 validation error 0.014545454545454545
epoch: 4
train error: 0.012165450121654502 train loss: 0.03226354505936901 validation error 0.014545454545454545
epoch: 5
train error: 0.019464720194647202 train loss: 0.04911834017730397 validation error 0.007272727272727273
epoch: 6
train error: 0.0072992700729927005 train loss: 0.02184005418647524 validation error 0.007272727272727273
epoch: 7
train error: 0.009732360097323601 train loss: 0.022395409482741567 validation error 0.007272727272727273
epoch: 8
train error: 0.00851581508515815 train loss: 0.020618752301523408 validation error 0.0145454545454